# **Proyecto 2 : Análisis de señales ECG**

## Andres Banquez- Melina Villada

1. De los artículos:

https://www.nature.com/articles/s41746-023-00966-w

https://www.nature.com/articles/s41598-020-59821-7

https://www.sciencedirect.com/science/article/pii/S2405844024032316

Para cada artículo hacer un diagrama de flujo de las metodologías usadas en los artículos (10%)

Explicar en sus palabras las metodologías desarrolladas (10%)



*Prediction of atrial fibrillation from at-home single-lead ECG signals without arrhythmias*

![Untitled Diagram (1).png](<attachment:Untitled Diagram (1).png>)